## Zero-Arb Study COW-Binance 

### Methodology


The objective is to compare COW settlement price with binance price. 

Use the timestamp of the COW trade transaction which is the block timestamp to compare with Binance price at that timestamp 

Binance allows querying of 1s candles with a limit of 1000 (~ 16.66 minutes)

To get more datapoints, will run this script with 15 min sleep time cycle for 3 days with the full script in `zero_arb_binance.py` 

### COW trades 

In [527]:
from datastreams.datastream import Streamer

import os
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [528]:
# instantiate Streamer class. Note that we need two separate streamer classes, otherwise the queries will be overwritten. 
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')
cow_ds2 = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

In [529]:
import time 

current_timestamp = int(time.time())
print(current_timestamp)

# we set a fixed query size number large enough to retrieve all trades within 15mins 
query_size = 1000

# add timestamp of 15min limit (15 min = 900 seconds)  
limit_timestamp = current_timestamp - 900 


1677495298


In [530]:
# query COW schema: trades
trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=query_size,
    orderBy='timestamp',
    where = {
    'timestamp_gt': limit_timestamp
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [531]:
print(f'query returned {len(trades_df)} rows')

query returned 14 rows


In [532]:
trades_df.head(5)
trades_df.shape
len(trades_df)

14

In [533]:
unique_tokens = trades_df['trades_buyToken_id'].unique()
print(unique_tokens)

['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
 '0xc0c293ce456ff0ed870add98a0828dd4d2903dbf'
 '0xdac17f958d2ee523a2206206994597c13d831ec7'
 '0x4f08705fb8f33affc231ed66e626b40e84a71870'
 '0x5f98805a4e8be255a32880fdec7f6728c6568ba0'
 '0xfc4913214444af5c715cc9f7b52655e788a569ed']


### Get symbols for Trades from `Token` schema 

In [534]:
# Set query size large enough to retrieve all tokens in the graph
query_size = 10000000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size,
)

# run query
token_df = cow_ds.runQuery(token_qp)


FIELD - tokens


In [535]:
# Check the number of tokens in the subgraph
token_df.shape

(2906, 13)

In [536]:
# check columns and types
token_df.dtypes

tokens_id                      object
tokens_address                 object
tokens_firstTradeTimestamp      int64
tokens_name                    object
tokens_symbol                  object
tokens_decimals                 int64
tokens_totalVolume             object
tokens_priceEth               float64
tokens_priceUsd               float64
tokens_numberOfTrades           int64
tokens_totalVolumeUsd         float64
tokens_totalVolumeEth         float64
endpoint                       object
dtype: object

In [537]:
# Check columns and types 
trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         uint64
trades_txHash            object
trades_settlement_id     object
trades_buyAmount         object
trades_sellAmount        object
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
trades_buyAmountEth     float64
trades_sellAmountEth    float64
trades_buyAmountUsd     float64
trades_sellAmountUsd    float64
endpoint                 object
dtype: object

In [538]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

merged_df.shape

(14, 29)

In [539]:
# Merge2 on buy token address.  y = buy token 
merged_df2 = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

merged_df2.shape

(14, 42)

In [540]:
merged_df2.dtypes

trades_id                        object
trades_timestamp                  int64
trades_gasPrice                   int64
trades_feeAmount                 uint64
trades_txHash                    object
trades_settlement_id             object
trades_buyAmount                 object
trades_sellAmount                object
trades_sellToken_id              object
trades_buyToken_id               object
trades_order_id                  object
trades_buyAmountEth             float64
trades_sellAmountEth            float64
trades_buyAmountUsd             float64
trades_sellAmountUsd            float64
endpoint_x                       object
tokens_id_x                      object
tokens_address_x                 object
tokens_firstTradeTimestamp_x      int64
tokens_name_x                    object
tokens_symbol_x                  object
tokens_decimals_x                 int64
tokens_totalVolume_x             object
tokens_priceEth_x               float64
tokens_priceUsd_x               float64


In [541]:
merged_df2 = merged_df2.rename(columns={
    "tokens_symbol_x": "sell_token_symbol",
    "tokens_symbol_y":"buy_token_symbol",
    "tokens_decimals_x": "sell_token_decimal", 
    "tokens_decimals_y": "buy_token_decimal"})

In [542]:
# filter out unnecessary columns
complete_trades_df = merged_df2[[
    'trades_id',
    'trades_timestamp', 
    'trades_gasPrice', 
    'trades_feeAmount',                 
    'trades_txHash',                    
    'trades_settlement_id',   
    'trades_sellAmount',
    'sell_token_decimal',
    'trades_buyAmount',   
    'buy_token_decimal',
    'trades_sellToken_id',              
    'trades_buyToken_id',               
    'trades_order_id',                  
    'sell_token_symbol',
    'buy_token_symbol'
    ]]



complete_trades_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol
0,0xd6a0e24af6872a1ab17e2b943e2883ba6cc8b6bb3ff7485e98eecc7e5d04d68effa820ee183f164ad57c7906e8796283052f753063fc88f8|0x6c056a649036ad8681f911c35667aca106a2675274cb0311ed12887af58e5627|2,1677494435,18200663901,13562380561,0x6c056a649036ad8681f911c35667aca106a2675274cb0311ed12887af58e5627,0x6c056a649036ad8681f911c35667aca106a2675274cb0311ed12887af58e5627,58323144882330,8,38640438435,6,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd6a0e24af6872a1ab17e2b943e2883ba6cc8b6bb3ff7485e98eecc7e5d04d68effa820ee183f164ad57c7906e8796283052f753063fc88f8,HEX,USDC
1,0xe26f0c15a7fec5cab64c31e342c56590aad00d975b5216951407a9206bbe5fd1d09e3d8cafe386b13792e08752e442669a571f4d63fc9017|0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40|132,1677494579,18691204356,340075793546743296,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,60000000000000000000,18,1563660346,6,0xd084944d3c05cd115c09d072b9f44ba3e0e45921,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe26f0c15a7fec5cab64c31e342c56590aad00d975b5216951407a9206bbe5fd1d09e3d8cafe386b13792e08752e442669a571f4d63fc9017,FOLD,USDC
2,0xe311ad3f8a4fec3c74c4dba10cdf21692e72c7c7d79e62c84a8d11d298b36fdb9e406b2c2021966f3983e899643609c45e3bbffe63fc9002|0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40|132,1677494579,18691204356,5406891163773634560,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,2172917549949158080436,18,7279335207,6,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe311ad3f8a4fec3c74c4dba10cdf21692e72c7c7d79e62c84a8d11d298b36fdb9e406b2c2021966f3983e899643609c45e3bbffe63fc9002,AURA,USDC
3,0x143731a30b1f283a6e1b13593ba4eb37c6c943b9fd523197e759932c970fd0123bc0cb287f74504347d50fe3ada6d90214e6f51263fc91e5|0x2f2b7c6d58ae45043976978b62463237e936d9e410a2dcc397f68b42d9ba0218|0,1677495143,16547708356,5892311159008494592,0x2f2b7c6d58ae45043976978b62463237e936d9e410a2dcc397f68b42d9ba0218,0x2f2b7c6d58ae45043976978b62463237e936d9e410a2dcc397f68b42d9ba0218,10000000000000000000000,18,33502984703,6,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x143731a30b1f283a6e1b13593ba4eb37c6c943b9fd523197e759932c970fd0123bc0cb287f74504347d50fe3ada6d90214e6f51263fc91e5,AURA,USDC
4,0xb7022cacc11b5d80d11c5b0e003befc4452f6be1d5843a0ebdd7a98d127402119e406b2c2021966f3983e899643609c45e3bbffe63fc9018|0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da|105,1677494627,17087313081,1488748257501125888,0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da,0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da,1247400631121225729923,18,8750125322,6,0xba100000625a3754423978a60c9317c58a424e3d,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xb7022cacc11b5d80d11c5b0e003befc4452f6be1d5843a0ebdd7a98d127402119e406b2c2021966f3983e899643609c45e3bbffe63fc9018,BAL,USDC
5,0xa16c2f0cffddf4e0ee3cb4c0cf3e0a8ab89bb0fb5e46f2ff5c4fbb8f69e1722b9e406b2c2021966f3983e899643609c45e3bbffe63fc902b|0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7|86,1677494663,17441573160,3489523010546406912,0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7,0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7,759528883106032018217,18,2266304739,6,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xa16c2f0cffddf4e0ee3cb4c0cf3e0a8ab89bb0fb5e46f2ff5c4fbb8f69e1722b9e406b2c2021966f3983e899643609c45e3bbffe63fc902b,LDO,USDC
6,0xe33a2365ef7491e41334d869f4fe7f294f88d602613f8895cd2cce1d3acdb37024d734a751

In [543]:
# Filter out addresses that do not have a symbol in the subgraph
complete_trades_df = complete_trades_df[complete_trades_df['buy_token_symbol'] != '']
complete_trades_df = complete_trades_df[complete_trades_df['sell_token_symbol'] != '']

complete_trades_df.shape

(14, 15)

In [544]:
# calculate buy and sell amounts from the correct decimal 
complete_trades_df['sell_amount'] = complete_trades_df.apply(lambda x: x['trades_sellAmount'] / (10**x['sell_token_decimal']), axis=1)
complete_trades_df['buy_amount'] = complete_trades_df.apply(lambda x: x['trades_buyAmount'] / (10**x['buy_token_decimal']), axis=1)

In [545]:
# calculate COW price defined sell amount / buy amount
complete_trades_df['cow_price'] = complete_trades_df['sell_amount'] / complete_trades_df['buy_amount']

In [546]:
complete_trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         uint64
trades_txHash            object
trades_settlement_id     object
trades_sellAmount        object
sell_token_decimal        int64
trades_buyAmount         object
buy_token_decimal         int64
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
sell_token_symbol        object
buy_token_symbol         object
sell_amount             float64
buy_amount              float64
cow_price               float64
dtype: object

### Binance price 

In [547]:
# Get list of available Binance pairs 

import requests 

host = "https://data.binance.com"
prefix = "/api/v3/ticker/price"
r = requests.get(host+prefix)
data = r.json()
binance_pairs = pd.DataFrame(data).loc[:,'symbol']

In [549]:
def query_binance(symbol: str, timestamp: int):

    """Queries the binance api to retrieve the price of symbol at a timestamp.
    Timestamp has to be within 1000seconds window ~ 16.66 mins"""

    host = "https://data.binance.com"
    prefix = "/api/v3/klines"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    payload = {'symbol':f'{symbol}','interval':'1s', 'limit':'1000'}
    r = requests.get(host+prefix,  params=payload, headers=headers)
    data = r.json()
    df = pd.DataFrame(data).filter(items=[0,4],axis=1)
    df.columns = ['timestamp', 'price']
    price = float(df.loc[df['timestamp'] == timestamp*1000, 'price'].iloc[0])
    print(f'price for {symbol} is {price}')
    return price 

# test
query_binance('ETHUSDT', current_timestamp)

price for ETHUSDT is 1638.07


1638.07

In [550]:
# Function to check if value string is in dataframe lis
if 'PERPUSDT' in binance_pairs.values:
    print('True')
else: 
    print('False')

True


In [551]:
def row_binance(sellTokenSymbol: str, buyTokenSymbol: str, timestamp: int):

	"""function to be used on the cow trades dataframe. Takes values from sellTokenSymbol 
	and buyTokenSymbol and timestamp to check the binance price for that trade. 

	Defines market_price as the price of sell token / price of buy token  
	Gets price of sell token and price of buy token from binance seperately

	Checks if token is USDT or a token that exists in Binance. Otherwise returns False 
	and not able to retrieve a price for that trade"""

	sell_pair = f'{sellTokenSymbol}USDT'
	buy_pair = f'{buyTokenSymbol}USDT'
	print('sell_pair: ', sell_pair)
	print('buy_pair: ', buy_pair)
	# retrieve sell token price  

	if sell_pair == 'USDTUSDT':
		sell_token_price = 1 
        
	elif sell_pair == 'USDCUSDT':
		sell_token_price = 1 
    
	elif sell_pair == 'WETHUSDT':
		sell_token_price = query_binance('ETHUSDT', timestamp)
        
	elif sell_pair == 'WBTCUSDT':
		sell_token_price = query_binance('BTCUSDT', timestamp)
        
	elif sell_pair in binance_pairs.values:
		sell_token_price = query_binance(sell_pair, timestamp)
        
	else: 
		return 'sell_unavailable' 

	# retrieve buy token price 
    
	if buy_pair =='USDTUSDT':
		buy_token_price = 1 
        
	elif buy_pair == 'USDCUSDT':
		buy_token_price = 1 
        
	elif buy_pair == 'DAIUSDT':
		buy_token_price = 1 / query_binance('USDTDAI')
    
	elif buy_pair == 'WETHUSDT':
		buy_token_price = query_binance('ETHUSDT', timestamp)
        
	elif buy_pair == 'WBTCUSDT':
		buy_token_price = query_binance('BTCUSDT', timestamp)
        
	elif buy_pair in binance_pairs.values:
		buy_token_price = query_binance(buy_pair, timestamp)
        
	else:
		return 'buy_unavailable'

	# calculate trade pair price 
	market_price = buy_token_price / sell_token_price 
	print(f'{sellTokenSymbol}/{buyTokenSymbol} binance price:', market_price)
	return market_price


#test 
row_binance('PERP', 'USDT', current_timestamp)

sell_pair:  PERPUSDT
buy_pair:  USDTUSDT
price for PERPUSDT is 0.96706
PERP/USDT binance price: 1.0340620023576614


1.0340620023576614

In [552]:
# Initialize a new column in the dataframe with value 0.
complete_trades_df['binance_price'] = 0.0
trades_analyzed = [] 
complete_trades_df = complete_trades_df.reset_index(drop=True)
complete_trades_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_price,binance_price
0,0xd6a0e24af6872a1ab17e2b943e2883ba6cc8b6bb3ff7485e98eecc7e5d04d68effa820ee183f164ad57c7906e8796283052f753063fc88f8|0x6c056a649036ad8681f911c35667aca106a2675274cb0311ed12887af58e5627|2,1677494435,18200663901,13562380561,0x6c056a649036ad8681f911c35667aca106a2675274cb0311ed12887af58e5627,0x6c056a649036ad8681f911c35667aca106a2675274cb0311ed12887af58e5627,58323144882330,8,38640438435,6,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd6a0e24af6872a1ab17e2b943e2883ba6cc8b6bb3ff7485e98eecc7e5d04d68effa820ee183f164ad57c7906e8796283052f753063fc88f8,HEX,USDC,5.832314e+05,3.864044e+04,15.093810,0.0
1,0xe26f0c15a7fec5cab64c31e342c56590aad00d975b5216951407a9206bbe5fd1d09e3d8cafe386b13792e08752e442669a571f4d63fc9017|0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40|132,1677494579,18691204356,340075793546743296,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,60000000000000000000,18,1563660346,6,0xd084944d3c05cd115c09d072b9f44ba3e0e45921,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe26f0c15a7fec5cab64c31e342c56590aad00d975b5216951407a9206bbe5fd1d09e3d8cafe386b13792e08752e442669a571f4d63fc9017,FOLD,USDC,6.000000e+01,1.563660e+03,0.038372,0.0
2,0xe311ad3f8a4fec3c74c4dba10cdf21692e72c7c7d79e62c84a8d11d298b36fdb9e406b2c2021966f3983e899643609c45e3bbffe63fc9002|0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40|132,1677494579,18691204356,5406891163773634560,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,0xac9abcfd7a05f3ed5662046e09bbaf483eafe786d9553de90634670f731d2f40,2172917549949158080436,18,7279335207,6,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe311ad3f8a4fec3c74c4dba10cdf21692e72c7c7d79e62c84a8d11d298b36fdb9e406b2c2021966f3983e899643609c45e3bbffe63fc9002,AURA,USDC,2.172918e+03,7.279335e+03,0.298505,0.0
3,0x143731a30b1f283a6e1b13593ba4eb37c6c943b9fd523197e759932c970fd0123bc0cb287f74504347d50fe3ada6d90214e6f51263fc91e5|0x2f2b7c6d58ae45043976978b62463237e936d9e410a2dcc397f68b42d9ba0218|0,1677495143,16547708356,5892311159008494592,0x2f2b7c6d58ae45043976978b62463237e936d9e410a2dcc397f68b42d9ba0218,0x2f2b7c6d58ae45043976978b62463237e936d9e410a2dcc397f68b42d9ba0218,10000000000000000000000,18,33502984703,6,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x143731a30b1f283a6e1b13593ba4eb37c6c943b9fd523197e759932c970fd0123bc0cb287f74504347d50fe3ada6d90214e6f51263fc91e5,AURA,USDC,1.000000e+04,3.350298e+04,0.298481,0.0
4,0xb7022cacc11b5d80d11c5b0e003befc4452f6be1d5843a0ebdd7a98d127402119e406b2c2021966f3983e899643609c45e3bbffe63fc9018|0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da|105,1677494627,17087313081,1488748257501125888,0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da,0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da,1247400631121225729923,18,8750125322,6,0xba100000625a3754423978a60c9317c58a424e3d,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xb7022cacc11b5d80d11c5b0e003befc4452f6be1d5843a0ebdd7a98d127402119e406b2c2021966f3983e899643609c45e3bbffe63fc9018,BAL,USDC,1.247401e+03,8.750125e+03,0.142558,0.0
5,0xa16c2f0cffddf4e0ee3cb4c0cf3e0a8ab89bb0fb5e46f2ff5c4fbb8f69e1722b9e406b2c2021966f3983e899643609c45e3bbffe63fc902b|0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7|86,1677494663,17441573160,3489523010546406912,0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7,0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7,759528883106032018217,18,2266304739,6,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x

In [553]:
# Loop through each row of the dataframe.
for i, row in complete_trades_df.iterrows():
    # Retrieve the trades_id, timestamp, sell token symbol, and buy token symbol from the row.
    trade_id = row[0]
    timestamp = row[1]
    sell_token_symbol = row[13]
    buy_token_symbol = row[14]
    timestamp_now = int(time.time())
    
    # Check first if timestamp is within 1000s of now to avoid panick. If it is old, then return 'old timestamp' 
    if abs(timestamp - timestamp_now) < 1000: 
        if trade_id not in trades_analyzed:
            # Use the pair_price_binance function to calculate the binance price and store it in the dataframe.
            complete_trades_df.iloc[i, 18] = row_binance(sell_token_symbol, buy_token_symbol, timestamp)
            trades_analyzed.append(trade_id)
        else: 
            complete_trades_df.iloc[i, 18] = 'repeat' 
            
    else:
        complete_trades_df.iloc[i, 18] = 'Timeout'
    
print('complete')

complete_trades_df


# Create a new column in the dataframe that stores the sell amount for the trade using binance price 
#df['sell_amount_with_binance'] = df['trades_buyAmount'] * df['binancePrice']


sell_pair:  HEXUSDT
buy_pair:  USDCUSDT
sell_pair:  FOLDUSDT
buy_pair:  USDCUSDT
sell_pair:  AURAUSDT
buy_pair:  USDCUSDT
sell_pair:  AURAUSDT
buy_pair:  USDCUSDT
sell_pair:  BALUSDT
buy_pair:  USDCUSDT
price for BALUSDT is 7.027
BAL/USDC binance price: 0.14230823964707556
sell_pair:  LDOUSDT
buy_pair:  USDCUSDT
price for LDOUSDT is 2.99
LDO/USDC binance price: 0.33444816053511706
sell_pair:  MCUSDT
buy_pair:  USDCUSDT
price for MCUSDT is 0.393
MC/USDC binance price: 2.544529262086514
sell_pair:  TONCOINUSDT
buy_pair:  USDCUSDT
sell_pair:  AURAUSDT
buy_pair:  WETHUSDT
sell_pair:  WETHUSDT
buy_pair:  AURAUSDT
price for ETHUSDT is 1635.47
sell_pair:  WETHUSDT
buy_pair:  LUSDUSDT
price for ETHUSDT is 1637.03
sell_pair:  ICEUSDT
buy_pair:  USDTUSDT
sell_pair:  DAIUSDT
buy_pair:  FLUTUSDT


IndexError: single positional indexer is out-of-bounds

In [554]:
# Filter out trades that do not have a symbol in the subgraph
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'repeat']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'timeout']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'buy_unavailable']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'sell_unavailable']


complete_trades_df.shape

(5, 19)

In [555]:
# Define a percentage difference function to get percentage difference between binance price and cow price 

def percentage_diff(col1, col2):
    """
    A function that calculates the percentage difference between two columns.
    """
    return ((col2.sub(col1)).div(col1)).mul(100)


# Create a new column in the dataframe that stores the percentage difference between the sell amount and the sell amount on Binance.
complete_trades_df['percentage_diff'] = percentage_diff(
    complete_trades_df['cow_price'], 
    complete_trades_df['binance_price']
)


In [556]:
# Filter out rows that have a difference higher than 50% as its likely to be a different token alltogether
# an example is LIT which is Litentry on Binance but Timeless on COW. Unfortunately binance api does not allow
# one to validate by token address only by string symbol 
complete_trades_df = complete_trades_df[abs(complete_trades_df['percentage_diff']) < 50]

In [557]:
complete_trades_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_price,binance_price,percentage_diff
4,0xb7022cacc11b5d80d11c5b0e003befc4452f6be1d5843a0ebdd7a98d127402119e406b2c2021966f3983e899643609c45e3bbffe63fc9018|0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da|105,1677494627,17087313081,1488748257501125888,0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da,0x3cd01d1fbd38ebe37cee2c47fc04b4bbb6a08422219c88534df1b5cfd306d9da,1247400631121225729923,18,8750125322,6,0xba100000625a3754423978a60c9317c58a424e3d,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xb7022cacc11b5d80d11c5b0e003befc4452f6be1d5843a0ebdd7a98d127402119e406b2c2021966f3983e899643609c45e3bbffe63fc9018,BAL,USDC,1247.400631,8750.125322,0.142558,0.142308,-0.17522
5,0xa16c2f0cffddf4e0ee3cb4c0cf3e0a8ab89bb0fb5e46f2ff5c4fbb8f69e1722b9e406b2c2021966f3983e899643609c45e3bbffe63fc902b|0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7|86,1677494663,17441573160,3489523010546406912,0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7,0x1c980bf65a4ae8aeef0b917fdb88e9b16bfc25a141a8d08d60b40a4428b1c6a7,759528883106032018217,18,2266304739,6,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xa16c2f0cffddf4e0ee3cb4c0cf3e0a8ab89bb0fb5e46f2ff5c4fbb8f69e1722b9e406b2c2021966f3983e899643609c45e3bbffe63fc902b,LDO,USDC,759.528883,2266.304739,0.335140,0.334448,-0.206369
6,0xe33a2365ef7491e41334d869f4fe7f294f88d602613f8895cd2cce1d3acdb37024d734a75166e66000c8d7f12a173e41358e3cc163fc9077|0xa7314b7a1d6a4d9153422bc5a5c7b36a0a7757ef7813ff1f0663645b0fb34652|146,1677494735,18084990756,17635599998195595264,0xa7314b7a1d6a4d9153422bc5a5c7b36a0a7757ef7813ff1f0663645b0fb34652,0xa7314b7a1d6a4d9153422bc5a5c7b36a0a7757ef7813ff1f0663645b0fb34652,5725049001025253039576,18,2245523070,6,0x949d48eca67b17269629c7194f4b727d4ef9e5d6,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xe33a2365ef7491e41334d869f4fe7f294f88d602613f8895cd2cce1d3acdb37024d734a75166e66000c8d7f12a173e41358e3cc163fc9077,MC,USDC,5725.049001,2245.523070,2.549539,2.544529,-0.196502


In [558]:
# Save the modified dataframe to a new csv file.
complete_trades_df.to_csv('zero_arb_binance_results.csv', index=False)